In [1]:
!pip3 install numpy
!pip3 install pandas
!pip3 install scikit-learn

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week03_ML/mushroom2020_dataset.csv')

In [4]:
df.isnull().sum()

id                               0
label                           60
cap-shape                        0
cap-surface                     27
bruises                         99
odor                            99
gill-attachment                 99
gill-spacing                   130
gill-size                      121
stalk-shape                    121
stalk-root                      31
stalk-surface-above-ring        31
stalk-surface-below-ring        31
veil-type                       62
ring-number                     62
ring-type                       62
spore-print-color               56
population                      56
habitat                         31
cap-color-rate                  27
gill-color-rate                121
veil-color-rate                 62
stalk-color-above-ring-rate     31
stalk-color-below-ring-rate     62
dtype: int64

Data Preparation

In [5]:
df.dropna(subset=['label'], inplace=True)
df.shape

(5764, 24)

In [6]:
drop_columns = ['id','gill-attachment', 'gill-spacing', 'gill-size','gill-color-rate','stalk-root', 'stalk-surface-above-ring',
 'stalk-surface-below-ring', 'stalk-color-above-ring-rate','stalk-color-below-ring-rate','veil-color-rate','veil-type']

df.drop(columns=drop_columns, inplace=True)
df.shape

(5764, 12)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5764 entries, 0 to 5823
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   label              5764 non-null   object 
 1   cap-shape          5764 non-null   object 
 2   cap-surface        5737 non-null   object 
 3   bruises            5665 non-null   object 
 4   odor               5665 non-null   object 
 5   stalk-shape        5643 non-null   object 
 6   ring-number        5702 non-null   object 
 7   ring-type          5702 non-null   object 
 8   spore-print-color  5708 non-null   object 
 9   population         5708 non-null   object 
 10  habitat            5733 non-null   object 
 11  cap-color-rate     5737 non-null   float64
dtypes: float64(1), object(11)
memory usage: 585.4+ KB


In [8]:
from sklearn.impute import SimpleImputer

# 1) select only numeric columns
numerical_cols = df.select_dtypes(include=['float64'])
num_imp = SimpleImputer(missing_values=np.NaN, strategy='mean')
df[numerical_cols.columns] = num_imp.fit_transform(numerical_cols)

# # 2) select only nominal columns
nominal_cols = df.select_dtypes(include=['object'])
nominal_cols.drop(columns=['label'], inplace=True)
cat_imp = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
df[nominal_cols.columns] = cat_imp.fit_transform(nominal_cols)

df
df.isnull().sum()

label                0
cap-shape            0
cap-surface          0
bruises              0
odor                 0
stalk-shape          0
ring-number          0
ring-type            0
spore-print-color    0
population           0
habitat              0
cap-color-rate       0
dtype: int64

In [9]:
df['cap-shape'].value_counts()

cap-shape
f    2382
x    2349
k     828
b     201
c       4
Name: count, dtype: int64

In [10]:
mapping_dict = {
    "label": {
        "p": 0,
        "e": 1,
    },
}

df = df.replace(mapping_dict)
df.label.value_counts()

/var/folders/vc/c5yh3t2s6tgc117z_ws0m1tw0000gn/T/ipykernel_8877/3373781591.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(mapping_dict)


label
0    3660
1    2104
Name: count, dtype: int64

In [11]:
dummy_df = pd.get_dummies(df[nominal_cols.columns], drop_first=True)
df = pd.concat([df, dummy_df], axis=1)
df.drop(nominal_cols.columns, inplace=True, axis=1)
df

,label,cap-color-rate,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,bruises_t,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,1.0,False,False,False,True,False,True,False,True,...,False,True,False,False,False,False,False,False,True,False
1,1,2.0,False,False,False,True,False,True,False,True,...,True,False,False,False,True,False,False,False,False,False
2,1,3.0,False,False,False,False,False,True,False,True,...,True,False,False,False,False,False,True,False,False,False
3,0,3.0,False,False,False,True,False,False,True,True,...,False,True,False,False,False,False,False,False,True,False
4,1,4.0,False,False,False,True,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819,1,1.0,False,False,True,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
5820,1,1.0,False,False,False,True,False,True,False,False,...,False,False,True,False,False,True,False,False,False,False
5821,1,1.0,False,True,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
5822,0,1.0,False,False,True,False,False,False,True,False,...,False,False,True,False,False,True,False,False,False,False


In [12]:
from sklearn.model_selection import train_test_split
y = df.pop('label')
x = df

x_train,x_test,y_train,y_test = train_test_split(x,y,stratify=y,test_size=0.2, random_state=2020)
print(x_train.shape)
print(x_test.shape)

(4611, 42)
(1153, 42)


Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2020)
param_grid = { 
    'min_samples_leaf':[2,5,10],
    'max_depth': [2,3,6],
    'n_estimators':[100,200],
    'criterion' :['gini', 'entropy']
}

In [14]:
from sklearn.model_selection import GridSearchCV
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(x_train, y_train)


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2020),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 6], 'min_samples_leaf': [2, 5, 10],
                         'n_estimators': [100, 200]})

In [15]:
CV_rfc.best_params_


{'criterion': 'gini',
 'max_depth': 6,
 'min_samples_leaf': 2,
 'n_estimators': 100}

In [16]:
import pickle
filename = 'model.sav'
pickle.dump(CV_rfc, open(filename, 'wb'))

Prediction and Evaluation

In [17]:
CV_rfc = pickle.load(open(filename, 'rb'))
CV_rfc

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2020),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 6], 'min_samples_leaf': [2, 5, 10],
                         'n_estimators': [100, 200]})

In [18]:
predictions = CV_rfc.predict(x_test)

In [19]:
from sklearn.metrics import classification_report,confusion_matrix

In [20]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       732
           1       1.00      0.99      0.99       421

    accuracy                           0.99      1153
   macro avg       1.00      0.99      0.99      1153
weighted avg       0.99      0.99      0.99      1153



In [21]:
print(confusion_matrix(y_test,predictions))

[[731   1]
 [  5 416]]
